In [5]:
import pandas as pd
import numpy as np
import sympy as sp

np.random.seed(123)


## Step 1: Get Raw Date

In [6]:
df = pd.read_csv('/Users/shawn/Desktop/碩士/碩二/國際併購/final_task/FF_simple.csv')  # change by yourself
df = df.tail(120)
df['Date'] = pd.to_datetime(df['年月'], format='%Y%m').dt.strftime('%Y-%m')
df = df[['Date', '無風險利率','市場風險溢酬']]
df.loc[:, df.columns != 'Date'] *= 0.01

# 生成資料
df['債券報酬率'] = np.random.normal(0.10, 0.20, len(df))
df['債券報酬率'] = df['債券報酬率'] * (0.2/df['債券報酬率'].std())
df['無風險利率'] = df['無風險利率'] * (0.02/df['無風險利率'].mean())
df.tail(3)

,Date,無風險利率,市場風險溢酬,債券報酬率
271,2024-08,0.02951,0.009623,0.029010
272,2024-09,0.02951,-0.005518,0.425897
273,2024-10,0.02951,0.024726,-0.264093


In [7]:
df.to_csv('/Users/shawn/Desktop/碩士/碩二/國際併購/final_task/weight.csv')

## Step 2 : 調整成 Excess Return

In [8]:
df['市場報酬率_adj'] = df['市場風險溢酬']  # 本來就是 Excess Return  形式
df['債券報酬率_adj'] = df['債券報酬率'] - df['無風險利率']
df.tail(3)

,Date,無風險利率,市場風險溢酬,債券報酬率,市場報酬率_adj,債券報酬率_adj
271,2024-08,0.02951,0.009623,0.029010,0.009623,-0.000499
272,2024-09,0.02951,-0.005518,0.425897,-0.005518,0.396387
273,2024-10,0.02951,0.024726,-0.264093,0.024726,-0.293602


## Step 3: 計算 V (conariance matrix)、$\mu$ (預期報酬率)

In [9]:
V = df[["市場報酬率_adj", "債券報酬率_adj"]].cov()
mean = df[["市場報酬率_adj", "債券報酬率_adj"]].mean()

V_matrix = V.values
mean_matrix = mean.values.reshape(-1, 1)

V_matrix, mean_matrix, np.linalg.inv(V_matrix)

(array([[ 0.00209741, -0.00033558],
        [-0.00033558,  0.04007316]]),
 array([[0.01109965],
        [0.06938055]]),
 array([[477.41773784,   3.99798418],
        [  3.99798418,  24.98783807]]))

## [Optional] 計算 t (benchmark of RRA) 

In [10]:
expected_rf = df['無風險利率'].mean()
expected_stock_mkt_retx = df['債券報酬率'].mean()
stock_mkt_vola = df['債券報酬率'].std()


t = sp.symbols('t', positive=True, real=True)
equation = sp.Eq(expected_rf, expected_stock_mkt_retx - (1 / (2 * t)) * (stock_mkt_vola ** 2))
t = sp.solve(equation, t)
t

[0.288265211844553]

## Step 4: 計算權重 $\theta$

In [11]:
V_matrix_inverse = np.linalg.inv(V_matrix)
theta = np.dot((t * V_matrix_inverse), mean_matrix)
theta = theta.astype(float)
theta

array([[1.60752599],
       [0.51254888]])

In [12]:
long_position = sum(theta[i][0].sum() for i in range(len(theta)) if theta[i][0] > 0)
short_position = sum(theta[i][0].sum() for i in range(len(theta)) if theta[i][0] < 0)
equity = 1
debt = long_position - equity
ck_lambda = debt/equity
round(ck_lambda, 2)

1.12

## 模擬

In [13]:
t = 0.25
V_matrix_inverse = 10000*np.array([[0.49, -0.39], [-0.39, 0.34]])
mean_matrix = 0.01*np.array([[1.92, 0.39]]).T

theta = np.dot((t * V_matrix_inverse), mean_matrix)
theta

array([[ 19.7175],
       [-15.405 ]])

In [14]:
long_position = sum(theta[i][0].sum() for i in range(len(theta)) if theta[i][0] > 0)
short_position = sum(theta[i][0].sum() for i in range(len(theta)) if theta[i][0] < 0)
equity = 1
debt = long_position - equity
ck_lambda = debt/equity
round(ck_lambda, 2)

18.72